---
title: IDs from ARTHEMIS
order: 12
---

## Background

ARTEMIS spacecrafts will be exposed in the solar wind at 1 AU during its orbits around the Moon. So it's very interesting to look into its data.

- For time inteval for THEMIS-B in solar wind, see [Link](https://omniweb.gsfc.nasa.gov/ftpbrowser/themis_b_sw.txt)
- For time inteval for THEMIS-C in solar wind, see [Link](https://omniweb.gsfc.nasa.gov/ftpbrowser/themis_c_sw.txt)


## Setup

Need to run command in shell first as `pipeline` is project-specific command

```{sh}
kedro pipeline create themis
```

To get candidates data, run `kedro run --from-inputs=jno.feature_1s --to-outputs=candidates.jno_1s`

In [ ]:
# | code-summary: import all the packages needed for the project
# | output: hide
# | export
import polars as pl
from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline

from ids_finder.pipelines.themis.mag import create_pipeline as create_mag_data_pipeline
from ids_finder.pipelines.themis.state import create_pipeline as create_state_data_pipeline
from ids_finder.pipelines.default.mission import create_combined_data_pipeline

In [ ]:
#| default_exp pipelines/themis/pipeline

## Processing the whole data

In [ ]:
#| export
from ids_finder.utils.basic import filter_tranges_df

def filter_sw_events(events: pl.LazyFrame, sw_state: pl.LazyFrame) -> pl.LazyFrame:
    
    start, end = sw_state.select(['start', 'end']).collect()
    sw_events = filter_tranges_df(events.collect(), (start, end))
    
    return sw_events

def create_sw_events_pipeline(
    sat_id,
    tau: int = 60,
    ts_mag: int = 1,
    
):
  
    ts_mag_str = f"ts_{ts_mag}s"
    tau_str = f"tau_{tau}s"
    
    node_filter_sw_events = node(
        filter_sw_events,
        inputs=[
            f"events.{sat_id}_{ts_mag_str}_{tau_str}",
            f"{sat_id}.inter_state_sw",
        ],
        outputs=f"events.{sat_id}_sw_{ts_mag_str}_{tau_str}"
        
    )

    nodes = [node_filter_sw_events]
    return pipeline(nodes)

In [ ]:
# | export
def create_pipeline(
    sat_id="THB",
    tau=60, # time window, in seconds
    ts_mag = 1, # time resolution of mag data, in seconds
) -> Pipeline:
    return (
        create_mag_data_pipeline(sat_id)
        + create_state_data_pipeline(sat_id)
        + create_combined_data_pipeline(sat_id)
        + create_sw_events_pipeline(sat_id, tau=tau, ts_mag= ts_mag)
    )